# Playing Time Influenced by Gender as a Predictor for Game Newsletter Subscription
By Group 23 (Aayush, Kathleen, QingYou, Gordon) 

## Introduction
#### Background:

Minecraft, the best-selling video game of all time, is a widely acclaimed action-adventure title first released in 2009. It's a game that is very accessable and popular amongst people of all demogrpahics. With over 300 million downloads, it has secured a significant place in gaming history. Due to minecrafts appeal, a study was conducted and funded by the Pacific Laboratory for Artificial Intelligence (PLAI), utilizing Minecraft’s broad popularity to explore gamer behavior. The research uses data collected through PLAICraft, a publicly accessible Minecraft server that runs directly in users' browsers, requiring no prior ownership of the game. Before joining the server, users complete a survey, which provides the datasets used in this project's analysis.

Online games increasingly rely on newsletter subscriptions as a tool to boost user engagement, communicate updates, and promote new features. Understanding what motivates players to subscribe provides valuable insights for developers and marketers aiming to tailor experiences to diverse player types. Given Minecraft’s massive reach and cultural impact, it serves as a compelling case study for this kind of behavioral analysis. Through PLAICraft, a browser-based Minecraft server created by the Pacific Laboratory for Artificial Intelligence (PLAI), researchers were able to collect detailed player data via a pre-game survey. This unique setup offers an ideal opportunity to examine patterns in subscription behavior across a broad and varied gaming audience. By analyzing these patterns, this study aims to uncover which player characteristics—such as gender, age, or play style—are most predictive of newsletter subscription.
#### Question

**General Question:**

What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types? (Question 1)

**Specific Interpretation:**

What is the relationship between total played hours and subscription class of a player? How does a player’s gender strengthen or weaken the predictive ability of average session time for subscription class?

#### Dataset Overview

To answer this question, we will use the players.csv file. The `players` dataset includes 196 observations and 7 variables describing player demographic information. The dataset size is sufficient for meaningful predictive modeling. The variables are given below.

| Variable | Type | Description |
|--------|----|-----------|
| `experience` | character | Player's skill level |
| `subscribe` | logical | Indicates whether player subscribed to a game-related newsletter |
| `hashedEmail` |character | Unique identifier for each player (hashed) |
| `played_hours` | numeric | The total hours the player has played |
| `name` | character | Player's name |
| `gender` | character | Gender |
| `Age` | numeric | Player's age |

Some brief summary statistics of the quantitative variables in the `players` dataset:
| Statistic | `played_hours` | `Age` |
| ---- | ---- | ---- |
| count | 196 | 194 |
| mean | 5.845918 | 20.52062 |
| median | 0.1 | 6.174667 |
| standard deviation | 28.35734 | 19 |
| min | 0 | 8 | 
| max | 223.1 | 50 | 

Some issues in the dataset are listed here:
* There might be some subjective inconsistencies with `experience` if it was self-reported.
* `Age` has 2 missing values.
* `experience` and `gender` should be factor variables.
* Distribution imbalances in variables like `subscribed`, `gender`, and `Age`.

Response variable:
* `subscribe`: Inidcates whether a player is subscribed to a game-related newsletter or not and is what we aim to predict.

Explanatory variables:

* `played hours`: In-game engagement time likely impacts the likelihood of players seeking out more game-information on newsletters.
* `gender`: For simplicity, we use a binary flag where one flag is male and the other is gender-diverse, which is an umbrella term that we will apply here to include women, non-binary people, two-spirit people, agender people, and those who responded 'other' in the survey. We will exclude those who answer 'prefer not to say'. We are interested on the possible effects this variable might have on the predictive capabilities of `played hours` on `subscribe` (If there is any).

## Methods and Results

To explore the impacts of gender on the predictive ability of `played hours`, we will start by training a classifier including data from all players. `played_hours` will be used as the predictive variable for the binary classification `subscribe` in a standard knn classification scheme. Then, we will split the data into two groups: one comprised of only male players, and one comprised of only gender diverse players. We will train one new classifier on each. This will leave us with three classifiers which we can then directly compare the skill of using standard classification metrics. 

#### Loading and Wrangling of Data

Getting necessary libraries and reading in players.csv dataset: 

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)

In [ ]:
players <- read_csv('https://raw.githubusercontent.com/kathleenramsey/dsci100_group23/main/Project%20Planning%20Players.csv')
players

Below we are creating three datasets (for our three models) from the original players dataset. 
1. The first dataset (players_full) has those who haven't specified a gender filtered out. `gender` variable modified, with "male" representing males and "gender diverse" representing female, non-binary, two-spirited, etc.
2. The second dataset (players_male) contains only males.
3. The third contains (players_gd) contains all listed genders except for males.
4. Each dataset only contains `played_hours`, `gender`, (explanatory) and `subscribe` (response) columns. Additionally, `subscribe` is converted into a factor variable for all three as well.

In [ ]:
players_full <- players |>
    select(subscribe, played_hours, gender) |>
    filter(gender != 'Prefer not to say') |>
    mutate(gender = if_else(gender == "Male", "male", "gender diverse")) |>
    mutate(subscribe = as.factor(subscribe))

players_male <- players |>
    select(subscribe, played_hours, gender) |>
    filter(gender == 'Male') |>
    mutate(subscribe = as.factor(subscribe))

players_gd <- players |>
    select(subscribe, played_hours, gender) |>
    filter(gender != 'Male', gender != 'Prefer not to say') |>
    mutate(subscribe = as.factor(subscribe))

### Relevant Statistics

Below we are getting some statistics on the variables from our players_full dataset we will use for our analysis. We can see roughly 3/4 of the players are subscribed to a game-related newspaper. More than half of our players are male and that there is a clear inbalance in `played_hours` as seen with the median.

In [ ]:
`subscription_%` <- players_full |>
    group_by(subscribe) |>
    summarize(count = n()) |>
    mutate(percentage = (count / sum(count)) * 100)
`subscription_%`

`gender_%` <- players_full |>
    group_by(gender) |>
    summarize(count = n()) |>
    mutate(percentage = (count / sum(count)) * 100)
`gender_%`

played_hours_summary <- players_full |>
    select(played_hours) |>
    summarize(mean = mean(played_hours, na.rm = TRUE),
              sd = sd(played_hours, na.rm = TRUE),
              median = median(played_hours, na.rm = TRUE),
              min = min(played_hours, na.rm = TRUE),
              max = max(played_hours, na.rm = TRUE))
played_hours_summary

### Visualizations

We'll explore some vizualisations to better understand our variables and their relationships with each other. 

**Figure 1: Play time distribution**

The histogram below shows what we mentioned earlier about the pretty substantial inbalance in `played_hours` amongst players. A huge majority of players have single-digit and near 0 hours played. Included in the plot is the distinction between those who are subscirbed and those who aren't subscribed to a game-related newspaper. We can see that ,unsurprisingly, most of the those not subscribed have lower or no hours.

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)

player_hist <- players_full |>
    ggplot(aes(x=played_hours, fill=subscribe)) +
    geom_histogram(binwidth=10) +
    labs(x='Play time (hours)',
         y='Number of players',
         fill='Subscribed to\ngame-related newsletter',
         title = 'Distribution of play time (Figure 1)') +
    theme(text = element_text(size = 15)) +
    scale_fill_brewer(palette = "Set2")
player_hist

**Figure 2: Distribution of gender groups (gender diverse and male)**

The bar graph below shows how much of males and gender diverse players there are. There are over half male players than there are of other genders. 

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)

gender_dist <- players_full |>
    ggplot(aes(x = gender, fill = gender)) +
    geom_bar(stat= 'count') +
    labs(x = 'Player gender', 
         y = 'Number of players', 
         fill = 'Player gender',
         title = 'Distribution of gender diverse/male (Figure 2)') +
    theme(text = element_text(size = 15)) +
    scale_fill_brewer(palette = "Set2")
gender_dist

**Figure 3: Subscription rate of gender groups (gender diverse and male)**

The bar graph below shows the rate of subscription for male players and players of other genders. We made the plot by converting the `subscribe` variable into a numerical (0/1) and used `mean()` to calculate the `subscribe_rate`. We can see that the rate at which gender diverse players subscribe are slightly higher (approx. 0.02) than the subscription rate of male players. This bar shows taht while the amount of gender diverse players are lesser, the rate of subscribing to a game related newsletter is much higher. 

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)

subscription_bar <- players_full |>
    mutate(subscribe = as.numeric(if_else(subscribe == "TRUE", TRUE, FALSE))) |>
    group_by(gender) |>
    summarize(subscribe_rate = mean(subscribe, na.rm = TRUE)) |>
    ggplot(aes(x = gender, y = subscribe_rate)) +
        geom_bar(stat = "identity", fill = "orange", alpha = 0.7) +
        labs(title = "Subscription rate of gender diverse/male (Figure 3)",
             x = "Gender",
             y = "Subscription rate") +
        theme(text = element_text(size = 15))
subscription_bar

### Data Analysis
Make training/testing splits. Train one classifier on data with all genders, choose optimum k

In [ ]:
# refer to worksheets/tutorials on classification ???
# use a CV (probably 4-fold) to do training/validation only on the players_full set, not the split sets
# i don't think that the split sets have enough data points to do a good train/test split and still get reliable results from CV
# so if we can find a k value with just the full set we can use that k value for the split sets too.

set.seed(23) 

full_split <- initial_split(players_full, prop = 0.75, strata = subscribe)
full_train <- training(full_split)
full_test <- testing(full_split)

k_vals <- tibble(neighbors = c(2:40))
knn_vfold <- vfold_cv(full_train, v = 4, strata = subscribe)

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_recipe <- recipe(subscribe ~ played_hours, data = full_train)

knn_workflow <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_tune)

knn_results <- tune_grid(
    knn_workflow,
    resamples = knn_vfold,
    grid = k_vals
)

knn_metrics <- knn_results |>
    collect_metrics() |>
    filter(.metric == "accuracy")

cross_val_plot <- knn_metrics |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(title = "accuracy vs. number of neighbors((Figure 4)",
         x = "nearest neighbors (k)",
         y = "accuracy")

print(cross_val_plot)

Figure 4 shows that the best k we can choose is between 30-40.

3. Using the optimal k found earlier, train classifiers on split data

In [ ]:
set.seed(23)

best_k <- 32

knn_model2 <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")


knn_workflow2 <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_model2)

knn_fit <- fit(knn_workflow2, data = full_train)

To build our final classification model, we used the K-Nearest Neighbors (KNN) algorithm with the optimal number of neighbors set to 32. This value was chosen because earlier cross-validation results showed that values of k between 32 and 40 produced the same accuracy. We set the KNN model to use rectangular weighting, meaning each of the nearest neighbors contributes equally to the classification decision. The model was then combined with a preprocessing recipe in a workflow to ensure the data is processed the same way each time the model is trained or used for prediction. Finally, we trained this workflow on the full training dataset so the model could learn the relationship between a player’s total played hours and their likelihood of subscribing to the newsletter. This trained model is used to evaluate prediction accuracy and explore how player characteristics, like gender, affect model performance.

4. Evaluate classifiers, gather skill metrics, compare and discuss

In [ ]:
# this is kind of discussion territory but we can see how the flow of the analysis goes
knn_preds <- predict(knn_fit, new_data = full_test)

knn_results <- bind_cols(knn_preds, full_test)

knn_metrics <- knn_results |>
  metrics(truth = subscribe, estimate = .pred_class)|>
  filter(.metric=="accuracy")

print(knn_metrics)


First, we use the `predict` function to make predictions using the trained KNN model (`knn_fit`) on the full test data (`full_test`). These predictions are stored in `knn_preds`. Then, we combine the predictions (`knn_preds`) with the actual test data (`full_test`) using `bind_cols`, creating the `knn_results` dataset.

Next, we calculate the evaluation metrics for the KNN model by using the `metrics` function. We specify that the true values are in the `subscribe` column and the predicted values are in the `.pred_class` column. We filter the results to only include the accuracy metric by using `filter(.metric == "accuracy")`.

Finally, we print the accuracy of the KNN model using `print(knn_metrics)`.

In [ ]:
# Train KNN model for male players and make predictions for male players in the test set
knn_fit_male <- knn_workflow2 |>
  fit(data = players_male)

knn_preds_male <- predict(knn_fit_male, new_data = players_male)

knn_results_male <- bind_cols(knn_preds_male, players_male)

# Calculate evaluation metrics for male players
knn_metrics_male <- knn_results_male |>
  metrics(truth = subscribe, estimate = .pred_class)|>
  filter(.metric=="accuracy")

print(knn_metrics_male)

We train the KNN model specifically for male players by using `knn_workflow2` and applying it to the `players_male` dataset. The trained model is stored in `knn_fit_male`.Then we use this model (`knn_fit_male`) to make predictions for the male players in the test set. We filter the `full_test` dataset to only include male players, then pass this filtered data to the `predict` function. The predictions are stored in `knn_preds_male`.
After that, we combine the predictions (`knn_preds_male`) with the corresponding actual test data for male players using `bind_cols`, creating the `knn_results_male` dataset.We then calculate the evaluation metrics for the KNN model on male players by using the `metrics` function. We specify that the true values are in the `subscribe` column and the predicted values are in the `.pred_class` column. We filter the results to only include the accuracy metric using `filter(.metric == "accuracy")`.Finally, we print the accuracy of the KNN model for male players using `print(knn_metrics_male)`.

In [ ]:
# Train the KNN model for gender-diverse players and make predictions for gender-diverse players in the test set
knn_fit_gd <- knn_workflow2 |>
  fit(data = players_gd)

knn_preds_gd <- predict(knn_fit_gd, new_data = players_gd)

knn_results_gd <- bind_cols(knn_preds_gd, players_gd)

# Calculate evaluation metrics for gender-diverse players
knn_metrics_gd <- knn_results_gd |>
  metrics(truth = subscribe, estimate = .pred_class)|>
  filter(.metric=="accuracy")

print(knn_metrics_gd)

In this step, we train the KNN model specifically for gender-diverse players by using `knn_workflow2` and applying it to the `players_gd` dataset. The trained model is stored in `knn_fit_gd`.

Next, we use this model (`knn_fit_gd`) to make predictions for gender-diverse players in the test set. We filter the `full_test` dataset to only include gender-diverse players, then pass this filtered data to the `predict` function. The predictions are stored in `knn_preds_gd`.

After that, we combine the predictions (`knn_preds_gd`) with the corresponding actual test data for gender-diverse players using `bind_cols`, creating the `knn_results_gd` dataset.

We then calculate the evaluation metrics for the KNN model on gender-diverse players by using the `metrics` function. We specify that the true values are in the `subscribe` column and the predicted values are in the `.pred_class` column. We filter the results to only include the accuracy metric using `filter(.metric == "accuracy")`.

Finally, we print the accuracy of the KNN model for gender-diverse players using `print(knn_metrics_gd)`.

This process is similar to the steps followed for male players, where we trained the model for male players, made predictions, and evaluated accuracy. Here, we are doing the same for gender-diverse players, ensuring that the model performance is evaluated for different player groups.

### Summary of Results (Accuracy):

1. **Accuracy for the full dataset (all players): 0.617** (61.7%)

2. **Accuracy or male players: 0.75** (75%)

3. **Accuracy for gender-diverse players: 0.77** (77%)

### Interpretation:
- The model performs reasonably well for the full dataset with an accuracy of 61.7%.
- The model performs better for male players, achieving an accuracy of 75%.
- The model works best for gender-diverse players, with an accuracy of 77%. This suggests that the model is more accurate at predicting subscription behavior for gender-diverse players compared to male players.

5.creates a visualization of the analysis


In [ ]:
options(repr.plot.height = 6, repr.plot.width = 10)
# Combine all player data for the plot
players_full <- players |>
  select(subscribe, played_hours, gender) |>
  filter(gender != 'Prefer not to say') |>
  mutate(gender = if_else(gender == "Male", "male", "gender_diverse")) |>
  mutate(subscribe = as.factor(subscribe))

# Create a histogram for all players
ggplot(players_full, aes(x = played_hours, fill = subscribe)) +
  geom_histogram(binwidth = 10, position = "dodge") +
  labs(
    title = "Distribution of Played Hours by Subscription Status(Figure 5)",
    x = "Played Hours",
    y = "Number of Players",
    fill = "Subscribed"
  ) +
  theme_minimal() +
  facet_wrap(~gender, scales = "free") 


### Explanation:
- **`geom_histogram()`**: Creates the histogram, where `binwidth` specifies the width of each bar in the histogram.
- **`position = "dodge"`**: This separates the bars for players who subscribed and those who did not.
- **`facet_wrap(~gender)`**: This creates separate histograms for male and gender-diverse players, allowing for easy comparison between the groups.
- **`labs()`**: Adds titles and axis labels to the plot.

This visualization will help us see how the distribution of played hours differs between subscribed and non-subscribed players and also how this pattern might vary between male and gender-diverse players.

## Discussion

### Summary of Findings:
The KNN models trained on different groups of players showed that total played hours is an important factor in predicting whether a player subscribes to a game-related newsletter. Players who spend more time playing the game are more likely to subscribe to the newsletter. This trend was observed in both male and gender-diverse players, though the model's performance varied across these groups.

- **For all players (combined):** The model had an accuracy of 61.7%. This suggests that total played hours can provide a moderate prediction of subscription likelihood across the full player base.
- **For male players:** The accuracy was 75%, indicating that played hours are an important predictor, but the model's performance is more reliable for male players than for the full dataset.
- **For gender-diverse players:** The accuracy was 77%, suggesting that played hours are a particularly strong predictor for this group. Gender-diverse players might have a clearer connection between their in-game time and subscription behavior compared to male players.

### Is this what you expected to find?

- **Expected Outcomes**: We thought that total played hours would be an important factor for predicting newsletter subscriptions because players who play more may want to get more information, like newsletters. This fits with our expectations, as time spent in the game likely shows an interest in staying informed about game updates, events, and other content.
  
- **Unexpected Outcomes**: The difference in model performance between male and gender-diverse players was surprising. We thought played hours would be useful for both groups, but the model did better for gender-diverse players. This means that their behavior in the game might match their subscription choices more clearly. It also shows that gender might affect not just the chance of subscribing but also how players react to game marketing. This could depend on their interests, gaming habits, or how much they engage with the game community.

The lower accuracy for male players suggests that there are other factors—possibly not in this dataset—that influence their decision to subscribe to newsletters. It may also mean that male players’ subscription behavior is less linked to in-game hours, or that including other features (like experience or age) would improve the model for this group.

In conclusion, the analysis showed that played hours is an important predictor for newsletter subscriptions, but it works better for gender-diverse players. This means that while played hours are helpful, exploring more features and improving the model (especially for male players) could make predictions more accurate for all players. Gender differences in how played hours predict subscription behavior could also be interesting to study further, as they may help us understand player behavior and improve marketing strategies.

### What impact could such findings have?

Our findings and results emphasize how crucial it is to take gender diversity into consideration when developing predictive models for user behavior analysis:

**Targeted Marketing Strategies:** Since we have shown that subscription behaviour is more predictable for gender-diverse and male players, companies can use this insight to create effective advertisements aimed at these players without over-targeting or overlooking certain groups.

**Gaming Product Design:** Game developers can explore why gender-diverse and male users like their game and newsletter and then use those discoveries to create more inclusive, engaging game features.

**Encourages Inclusive Data Science Methods:** Our research highlights the importance of including a wide range of demographics groups when collecting data in order to gain a more complete picture of user behaviour.

**Discovering Hidden Model Biases:** The discrepancy between our overall accuracy (61.7%), male-only accuracy (75%), and gender-diverse accuracy (77%) demonstrates that certain subgroups may underperform or excel with specific models. This shows that data analysis should always involve subgroup analysis when evaluating accuracy of a given model.

### What future questions could this lead to?

Our findings and results open the door to a variety of thought-provoking questions that require further analysis to answer such as:

1. Why is the subscription predictive accuracy so low for the complete dataset? If we used models for each subgroup of gender, would all of them produce greater accuracies?

2. What reasons do different genders value the game and newsletter for? Is it the actual content of the game? Are certain genders more likely to develop an emotional connection? For instance, some players may feel a stronger emotional attachment because of the inclusivity and representation in the game and the community. Are certain genders more likely to rely on the newsletter or external sources for updates on the game?

3. How might other factors and their interactions with gender affect subscription predictability? For example, what would change if we also used experience or age as predictors?

4. How would our results change in larger datasets or different types of games?
